<span style="color:red">Abgegeben von (Name, Vorname):</span> 
Musterfrau, Marina

In [ ]:
import nltk

Immer griffbereit:
- Website: https://www.nltk.org/
- Buch: https://www.nltk.org/book/ch08.html
- Module: https://www.nltk.org/py-modindex.html
- Beispiele: http://www.nltk.org/howto/

# Tiefe Satzanalyse

In den letzten Sitzungen haben wir Methoden kennengelernt, Chunks zu identifizieren und zu klassifzieren. Das Chunking ist aber nur eine sehr oberflächliche Form der Satzanalyse (deshalb **Shallow Parsing**), denn die Chunks sind ja nicht-rekursiv: 

    (S
      (NP We/PRP)
      see/VBD
      (NP the/DT yellow/JJ dog/NN) 
      (PP of/IN)
      (NP the/DT neighbor/NN))
      
Man kann also manche Beziehungen nicht ausdrücken, wie zum Beispiel hier die Zusammengehörigkeit von `the/DT yellow/JJ dog/NN of/IN the/DT neighbour/NN` als Objekt von `see` und gleichzeitig, dass `of/IN the/DT neighbour/NN` ein zusammenhängender Bestandteil dieses komplexen Objekts ist.

Statt dieser flachen Struktur hätten wir also gerne eine "tiefe" NP:

    (S
      (NP We/PRP)
      see/VBD
      (NP the/DT yellow/JJ dog/NN 
         (PP of/IN
            (NP the/DT neighbor/NN))))
            
Ein anderes sehr häufiges Beispiel für Rekursion betrifft in Sätze eingebettete Sätze:

    (S
       (NP Andre/NNP)
       said/VBD
      (S
        (NP the/DT Jamaica/NNP Observer/NNP)
        reported/VBD
        (S 
           (NP Usain/NNP Bolt/NNP) 
           broke/VBD 
           (NP the/DT record/NN))))

In diesem Notebook werden wir uns mit Verfahren beschäftigen, mit denen solche rekursiven, tiefen Strukturen erzeugt werden können.

## Kaskadierte Chunker

Eine recht simple Möglichkeit, rekursive Klammerstrukturen zu erzeugen, besteht darin, den Chunker auf bereits gechunkte Sätze anzuwenden. Erst dann entfalten rekursive Regeln wie `S:  {<NP><VB.*><NP|PP|S>+$}` ihre ganze Wirkung. 

Um zu erreichen, dass solche rekursive Regeln wiederholt angewendet werden, muss man bei der Instanziierung von `RegexpParser` den Parameter `loop` auf einen Wert $\geq 2$ setzen:  

In [ ]:
grammar = r"""
  NP: {<DT|JJ|NN.*|PRP>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN><NP>}               # Chunk prepositions followed by NP
  S:  {<NP><VB.*><NP|PP|S>+$}  # Chunk verbs and their arguments
  """
cp = nltk.RegexpParser(grammar,loop=3)

sentence = [("I","PRP"), ("think","VBD"),  ("Andre","NNP"), ("said","VBD"), ("the","DT"), ("Jamaica","NNP"), ("Observer", "NNP"), ("reported","VBD"), ("Usain","NNP"), ("Bolt","NNP"), ("broke","VBD"), ("the","DT"), ("record", "NN")]

print(cp.parse(sentence))

An diesem Beispiel sieht man aber auch sehr deutlich einen Nachteil von kaskadierten Chunkern (zumindest in der NLTK-Implementierung mit `RegexpParser`): Man muss die Anzahl der Wiederholungen und damit auch den maximalen Grad der Rekursion explizit vorgeben.  

## Kontextfreie Grammatiken

Im Unterschied zu kaskadierten Chunkern muss man bei [kontextfreien Grammatiken (CFG)](https://de.wikipedia.org/wiki/Kontextfreie_Grammatik) weder die Reihenfolge der Regelanwendung noch die Anzahl der Loops festlegen.   

Für die Implementierung und Verarbeitung von CFGs stellt NLTK das Modul [`nltk.grammar.CFG`](https://www.nltk.org/api/nltk.html?highlight=cfg#nltk.grammar.CFG) bereit.

CFGs bestehen im Wesentlichen aus Ersetzungsregeln der Form `A -> B C ...` und einem Startsymbol. Die Regeln können mit Hilfe der Methode `fromstring` eingegeben werden, wobei als Startsymbol `S` angenommen wird.

Als Nichtterminale dienen üblicherweise die folgenden syntaktischen Phrasentypen (und deren "Köpfe"): 

| **Symbol** | **Meaning**             | **Example**          |
| :----- | :------------------- | :--------------- |
| S      | sentence             | *the man walked*   |
| NP     | noun phrase          | *a dog*            |
| VP     | verb phrase          | *saw a park*       |
| PP     | prepositional phrase | *with a telescope* |
| AP     | adjective phrase     | *very good*        |
| Det    | determiner           | *the*              |
| N      | noun                 | *dog*              |
| V      | verb                 | *walked*           |
| P      | preposition          | *in*               |
| A      | adjective            | *yellow*           |

Für den Satz *We see the yellow dog of the neighbour* könnten wir beispielsweise die folgende CFG angeben, wobei wir der Einfachheit halber die POS-Tags weglassen.

In [ ]:
first_grammar = nltk.grammar.CFG.fromstring("""
S -> NP V NP
NP -> Det A N PP | Det N | N
PP -> P NP
Det -> 'the' 
N -> 'We' | 'dog' | 'neighbour' 
A -> 'yellow'
P -> 'of'
V -> 'see'
""")

print(first_grammar)

Diese CFG können wir dann einem [`ChartParser`](https://de.wikipedia.org/wiki/Chart-Parser) übergeben, der damit den Satz *We saw the yellow dog of the neighbour* mit einer rekursiven NP analysiert.

In [ ]:
first_parser = nltk.ChartParser(first_grammar)

sent = nltk.word_tokenize("We see the yellow dog of the neighbour")

for tree in first_parser.parse(sent):
    print(tree)

Übrigens kann man mit `draw()` die Phrasenstruktur als Baum darstellen.

In [ ]:
#for tree in first_parser.parse(sent):
#    tree.draw()

Die damit generierbaren Sätze lassen sich folgendermaßen erzeugen, wobei die Parameter `n` und `depth` möglichst niedrig gewählt werden sollten ;-)

In [ ]:
from nltk.parse.generate import generate

for sent in generate(first_grammar,n=10,depth=4):
    print(sent)

### Ungrammatische Sätze

Offensichtlich können wir mit `first_grammar` auch viele "ungrammatische" Sätze parsen/erzeugen, z.B. *the We see dog* oder *the dog see the dog*. Um dies zu vermeiden, müssen die Nichtterminale weiter differenziert und die Regeln angepasst werden. 

Man beachte, dass der Anspruch bei solchen CFG nicht nur darin besteht, eine vernünftige Analyse herauzubekommen, sondern auch genau die Menge der **grammatisch wohlgeformten Sätze** einer Sprache zu erzeugen.

Um Sätze wie *the dog see the dog* zu verhindern und die Numeruskongruenz zwischen Subjekt und finitem Verb sicherzustellen, müsste man beispielsweise eine Unterscheidung wie `S -> NP_sg V_sg NP` und `S -> NP_pl V_pl NP` einführen.  

Dies lässt sich eleganter mit [Merkmalsstrukturen](https://www.nltk.org/book/ch09.html) umsetzen, mit denen die beiden Regeln zusammengefasst werden könnten: `S -> NP[NUM=?n] V[NUM=?n] NP`.

### Zusätzliche Rekursion

Man beachte außerdem, dass wir bei der Regelspezifikation nicht mehr auf **reguläre Ausdrücke** zurückgreifen können wie noch beim Chunk-Parser. Es ist also nicht möglich, NP-Regel kompakt so anzugeben: `NP -> (Det)? (A)* N`. <span style="color:red">(Frage am Rande: Warum ist das bei CFGs nicht zu empfehlen?)</span>

Um an dieser Stelle eine beliebige Kette von Adjektiven zuzulassen, muss man weitere, zum Teil rekursive Regeln einführen, z.B. `NP -> Det N'` und `N' -> A N'` und `N' -> N`.    

### <span style="color:red">Aufgaben I: Kontextfreie Grammatik</span>

<span style="color:red">A1:</span> Implementieren Sie eine kontextfreie Grammatik mit `nltk.grammar.CFG.fromstring`, die den Satz *I think Andre said the Jamaica Observer reported Usain Bolt broke the record* entsprechend der folgenden Struktur erzeugt: 

    (S
      (NP I/PRP)
      think/VBD
      (S
        (NP Andre/NNP)
        said/VBD
        (S
          (NP the/DT Jamaica/NNP Observer/NNP)
          reported/VBD
          (S (NP Usain/NNP Bolt/NNP) broke/VBD (NP the/DT record/NN)))))
          
Nutzen Sie dabei die POS-Tags als Präterminale.

In [ ]:
# Lösung A1:

bolt_grammar = nltk.grammar.CFG.fromstring("""

S -> Epsilon

""")

bolt_parser = nltk.ChartParser(bolt_grammar)
sent = nltk.word_tokenize("I think Andre said the Jamaica Observer reported Usain Bolt broke the record")

for tree in bolt_parser.parse(sent):
    print(tree)

### Ambiguität

Anders als beim Chunking mit dem `RegexpParser` sind beim CFG-Parsing oft mehrere Analysen für einen Satz möglich. Dieses Phänomen, dass sprachliche Zeichen mehrere Analysen erhalten können, nennt man in der Linguistik **Ambiguität**.

Das NLTK-Buch zitiert ein klassisches Beispiel für **syntaktische oder strukturelle Ambiguität** (von Groucho Marx):

> While hunting in Africa, I shot an elephant <span style="text-decoration:underline">in my pajamas</span>.
> How he got into my pajamas, I don't know.

Der Witz entsteht durch die Ambiguität der PP *in my pajamay*, die sich sowohl auf *I* als auch auf *an elephant* beziehen kann. Man nennt das in der Literatur auch **PP-Attachment Ambiguity**. Nun ist die Kombination mit *I* sehr viel plausibler, wenn man das Weltwissen berücksichtigt. Aus dem Kontext wird aber klar, das eigentlich die (sehr viel unplausiblere) Kombination mit *an elephant* gemeint ist.

An diesem Beispiel wird deutlich, dass Ambiguität an unvermuteten Stellen auftauchen kann. Außerdem kann auch die unplausibelste Bedeutung intendiert sein (und sei es nur zum Spaß). Eine Grammatik muss also prinzipiell alle Bedeutungen erzeugen/analysieren können. 

Im NLTK-Buch wird diese Ambiguität so modelliert, dass die PP *in my pajamay* entweder an der VP oder an der Objekt-NP "angehängt" wird:

In [ ]:
groucho_grammar = nltk.grammar.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

parser = nltk.ChartParser(groucho_grammar)
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

for tree in parser.parse(sent):
    print(tree)

In [ ]:
# for tree in list(parser.parse(sent)):
#     tree.draw()

Die strukturelle Ambiguität (und natürlich auch die lexikalische Ambiguität) ist eine große Herausforderung für CFG-Parser (und für NLP-Verfahren im Allgemeinen). Durch die freie Kombinatorik der Möglichkeiten wächst der Suchraum abhängig von der Satzlänge oft sehr stark. Hier muss der Parser einen möglichst ressourcenschonenden Weg finden.  

### CFG-Parser

NLTK enthält "Apps" für verschiedene Parsingansätze, mit denen man die Ansätze interaktiv erkunden kann.

#### Recursive Descent Parser

Ein einfacher [**(LL-)Top-Down-Parser**](https://de.wikipedia.org/wiki/LL-Parser), der ausgehend vom Startsymbol die Ausgabe von links nach rechts zu erzeugen versucht, ohne die Ausgabe bei der Wahl der Produktionen zu berücksichtigen. Bei der Verwendung linksrekursiver Regeln (`NP -> NP PP`) kommt es zu Endlosschleifen. 

In [ ]:
nltk.app.rdparser()

#### Shift-Reduce Parser

Ein einfacher [**(LR-)Buttom-Up-Parser**](https://de.wikipedia.org/wiki/LR-Parser), der ausgehend von den Terminalen das Startsymbol zu erreichen versucht. Die Produktionen werden also quasi umgedreht, so dass die Symbole auf der rechten Seite (RHS) durch die Symbole auf der linken Seite (LHS) ersetzt werden. Endlosschleifen ergeben sich bei rekursiven unären Produktionen (`NP -> NP`). Durch den Verzicht aufs Backtracking benötigt der SR-Parser nur lineare Zeit, kann dafür aber auch nicht alle kontextfreie Sprachen erkennen.

In [ ]:
nltk.app.srparser()

#### Chart Parsing

Schließlich können diese (und andere) Parsingstrategien mit einem [Speichermechanismus](https://de.wikipedia.org/wiki/Chart-Parser) kombiniert werden, den man **Dynamic Programming** nennt. Das bedeutet im Grunde nichts anderes, als dass Zwischenergebnisse dauerhaft in einer Tabelle ("Chart") gespeichert und bei der Bearbeitung alternativer Parsingwege wiederverwendet werden können. 

In [ ]:
nltk.app.chartparser_app.app()

## Dependenzstrukturen

Bisher haben wir uns mit sogenannten **Konstituenten** (aka Chunks oder Phrasen) beschäftigt, d.h. mit Gruppierungen von Worttoken, die Teil-Ganzes-Beziehungen widerspiegeln.

In der **Dependenzgrammatik** wird dagegen nur mit Beziehungen (sogenannten **Dependenzen**) zwischen Worttoken gearbeitet; Konstituenten spielen dort allenfalls indirekt eine Rolle. Dies hat zur Folge, dass die syntaktischen Repräsentationen sparsamer sind, da es dort keine abstrakten Knoten gibt.  

![](https://www.nltk.org/images/depgraph0.png)

Die Dependenzen sind darüber hinaus funktional eindeutiger beschriftet. Beides hat gewisse Vorteile, so dass sich Dependenzstrukturen zu einem wichtigen Element in der NLP entwickelt haben. 

Auf der anderen Seite ist es oft nicht einfach, zu entscheiden, in welche Richtung die Dependenzen zeigen, d.h. welches Wort der Kopf ist und welches der Dependent. Hier lohnt ein Blick auf die Guidelines der [UD-Initiative](https://universaldependencies.org/guidelines.html).

NLTK bietet leider nur eine rudimentäre Unterstützung von Dependenzstrukturen und Dependenzparsern. Wir können immerhin Dependenzgrammatiken wie CFGs aufschreiben (was kein Fehler ist) und einem Dependenzparser übergeben: 

In [ ]:
groucho_dep_grammar = nltk.DependencyGrammar.fromstring("""
'shot' -> 'I' | 'elephant' | 'in'
'elephant' -> 'an' | 'in'
'in' -> 'pajamas'
'pajamas' -> 'my'
""")

print(groucho_dep_grammar)

In [ ]:
pdp = nltk.ProjectiveDependencyParser(groucho_dep_grammar)
sent = nltk.word_tokenize('I shot an elephant in my pajamas')
trees = pdp.parse(sent)
for tree in trees:
    print(tree)

Leider sehe ich nicht, wie man hier noch Dependenzlabel hinzufügen kann.

## Stochastische/Gewichtete CFGs

Zum Schluss noch der Hinweis, das angesichts der großen Anzahl der möglichen Parsinglösungen für einen Satz (Stichwort Ambiguität) nicht immer alle Lösungen ausgereichnet werden können. Zudem will man gemeinhin nicht 253 irgendwie mögliche Analysen erhalten, sondern die eine plausible. 

Um das zu erreichen, sind Verfahren zur Gewichtung von Analysen notwendig. Eine naheliegende Möglichkeit besteht darin, die Produktionen einer CFG mit Wahrscheinlichkeitsmaßen zu versehen (**Probabilistic CFG**). Dabei muss eigentlich nur darauf geachtet werden, dass die Wahrscheinlichkeiten der Produktionen mit derselben LHS in der Summe $1$ ergeben.  

In [ ]:
# from NLTK
grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.3]
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> 'telescopes'       [0.8]
    NP   -> 'Jack'             [0.2]
    """)

In [ ]:
viterbi_parser = nltk.ViterbiParser(grammar)

for tree in viterbi_parser.parse(['Jack', 'saw', 'telescopes']):
    print(tree)


Die Wahrscheinlichkeitsmaße sowie die Grammatik werden üblicherweise direkt aus vorannotierten Corpora (sogenannte Baumbanken) induziert. Daneben gibt es aber auch große, handgefertigte Grammatiken. 